# **Простой бот** (без памяти)

Используется `Qwen3-8b:free` (по API с OpenRouter)

Инструменты: **parse_user_query** **get_current_time**, **get_weather**

Структура: parse_user_query использует LLM Qwen для обработки запроса пользователя, и выводит JSON следующего формата: 

`{`

    "action": "get_weather" или "get_time", иначе null,

    "date": дата в формате YYYY-MM-DD или null, если не указана,

    "location": город как строка (только на русском языке) или null, если не указан

`}`

На основе данных с парсера, применяем get_current_time либо get_weather

P.S. Бот работает с русским языком. Конечно, парсер может корректно обработать запросы на английском (причем успешно, есть примеры ниже), но ответы бот выдает только на русском, пользователь не поймет. В теории модель можно доработать, например, чтобы парсер выводил в JSON еще и language (русский или английский). А затем построить функцию с if.

### **Создание БД**

In [2]:
from modules.cache import init_db

In [3]:
init_db()

### **Примеры работы инструментов**
#### *Парсер*
Обрабатывает вопрос пользователя и создает json-файл, содержащий action (get_time для времени, get_weather для погоды), date для даты в формате YYYY-MM-DD, location для города.

In [1]:
from modules.llm_parser import parse_user_query

In [5]:
print(parse_user_query("Сколько время?"))

{'action': 'get_time', 'date': None, 'location': None}


In [3]:
print(parse_user_query("Сколько планет в солнечной системе?"))

{'action': None, 'date': None, 'location': None}


In [10]:
print(parse_user_query("Какая погода была в Миассе 25 октября 2003 года?"))

{'action': 'get_weather', 'date': '2003-10-25', 'location': 'Миасс'}


In [2]:
print(parse_user_query("Tell me the weather in LA"))

{'action': 'get_weather', 'date': None, 'location': 'Лос-Анджелес'}


In [3]:
print(parse_user_query("Hello bro, how are you?"))

{'action': None, 'date': None, 'location': None}


#### *Время*
Функция получает на вход пустое значение для вывода системного времени, либо конкретный город, для вывода текущего времени в указанном городе

In [1]:
from modules.time import get_current_time

In [12]:
print(get_current_time())

Сейчас 13:23:06 (по системному времени)


In [6]:
print(get_current_time("Нью-Йорк"))

Сейчас 02:29:27 в городе Нью-Йорк


In [7]:
print(get_current_time("Москва"))

Сейчас 09:30:02 в городе Москва


### **Погода**

Функция получает на вход название города и дату в формате YYYY-MM-DD

In [7]:
from modules.weather import get_weather

In [2]:
print(get_weather("Миасс", "2003-10-25"))

Температура в этот день была -5.9°C ночью и -3.9°C днем.


In [3]:
print(get_weather("Северск", "2025-06-22"))

Температура в этот день была 15.6°C ночью и 25.6°C днем.


***
## **Бот + диалог**

In [1]:
from modules.llm_parser import parse_user_query
from modules.time import get_current_time
from modules.weather import get_weather

In [5]:
print(
    """
    🤖 Привет! Я бот, который подскажет тебе текущее время и погоду. \n 
    Если захочешь завершить диалог, напиши "стоп".\n
    """
    )

while True:
    user_input = input("Пользователь: ")
    print (f"Пользователь: {user_input}")
    
    if user_input.strip().lower() in {"выход", "exit", "quit", "стоп", "stop"}:
        print("Бот: До свидания!")
        break

    parsed = parse_user_query(user_input)
    action = parsed.get("action")
    date = parsed.get("date")
    location = parsed.get("location")

    if action == "get_time":
        response = get_current_time(location)
        print(f"Бот: {response}")

    elif action == "get_weather":
        if not date or not location:
            print("Бот: Не хватает данных, пожалуйста, укажите полный запрос, например: «Какая погода была в Москве 25 января 2025 года».")
        else:
            response = get_weather(location, date)
            print(f"Бот: {response}")

    else:
        print("Бот: Я могу рассказать только про погоду и текущее время.")


    🤖 Привет! Я бот, который подскажет тебе текущее время и погоду. 
 
    Если захочешь завершить диалог, напиши "стоп".

    
Пользователь: Привет, что ты умеешь?
Бот: Я могу рассказать только про погоду и текущее время.
Пользователь: Сколько будет 100-10?
Бот: Я могу рассказать только про погоду и текущее время.
Пользователь: Сколько сейчас времени?
Бот: Сейчас 21:18:56 (по системному времени)
Пользователь: А сколько времени в Москве?
Бот: Сейчас 21:19:12 (по системному времени)
Пользователь: Какая погода в Томске
Бот: Не хватает данных, пожалуйста, укажите полный запрос, например: «Какая погода была в Москве 25 января 2025 года».
Пользователь: Какая погода была в Томске 15 апреля 2025 года?
Бот: Температура в этот день была -4.2°C ночью и 3.9°C днем.
Пользователь: стоп
Бот: До свидания!
